In [ ]:
"""
Pytorch safe globals için merkezi sağlayıcı modül.
Bu dosya tüm projenin tek bir yerden safe globals kaydı yapmasını sağlar.
"""
import torch.serialization
from ultralytics.nn.tasks import DetectionModel
from typing import Dict, Set, Optional

# Kayıt edilmiş global'leri takip eder
_REGISTERED_GLOBALS: Set[str] = set()

# Kaydedilecek tüm sınıflar burada tanımlanır
SAFE_GLOBALS: Dict[str, object] = {
    "ultralytics.nn.tasks.DetectionModel": DetectionModel
}

def register_torch_safe_globals() -> bool:
    """
    Tüm proje için tek bir yerden PyTorch güvenli globals kaydı yapar.
    Her çağrıda yalnızca bir kez kaydeder.

    Returns:
        bool: Kayıt yapıldıysa True, zaten yapılmışsa False
    """
    global _REGISTERED_GLOBALS

    # add_safe_globals fonksiyonunu kontrol et
    add_safe_fn = getattr(torch.serialization, "add_safe_globals", None)
    if not callable(add_safe_fn):
        print("[WARNING] torch.serialization.add_safe_globals not found; "
              "could not add safe globals.")
        return False

    # Her bir global için tek tek kayıt yapalım
    registered = False
    for class_path, class_obj in SAFE_GLOBALS.items():
        if class_path not in _REGISTERED_GLOBALS:
            # İlgili sınıfı güvenli global olarak ekle
            single_class_dict = {class_path: class_obj}
            try:
                add_safe_fn(single_class_dict)
                _REGISTERED_GLOBALS.add(class_path)
                registered = True
                print(f"[INFO] Registered safe global: {class_path}")
            except Exception as e:
                print(f"[ERROR] Failed to register {class_path}: {str(e)}")

    return registered